In [11]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

In [12]:
tf.__version__

'2.8.0'

In [13]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        'gender_dataset/train',
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

Found 18965 images belonging to 2 classes.


In [14]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_set = test_datagen.flow_from_directory(
    'gender_dataset/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

Found 4743 images belonging to 2 classes.


In [15]:
cnn = tf.keras.models.Sequential()

In [16]:
#input layer
cnn.add(tf.keras.layers.Conv2D(filters = 36, kernel_size = 3, activation = 'relu', input_shape = (128, 128, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2))

cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu', input_shape = (128, 128, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2))

cnn.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu', input_shape = (128, 128, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2))

cnn.add(tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, activation = 'relu', input_shape = (128, 128, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2))

cnn.add(tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, activation = 'relu', input_shape = (128, 128, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))

cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [17]:
cnn.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = 'accuracy')

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks = [early_stopping])

Epoch 1/25
593/593 [==============================] - 351s 591ms/step - loss: 0.5982 - accuracy: 0.6593 - val_loss: 0.4406 - val_accuracy: 0.7984
Epoch 2/25
593/593 [==============================] - 330s 556ms/step - loss: 0.4130 - accuracy: 0.8151 - val_loss: 0.3280 - val_accuracy: 0.8649
Epoch 3/25
593/593 [==============================] - 329s 555ms/step - loss: 0.3458 - accuracy: 0.8503 - val_loss: 0.3699 - val_accuracy: 0.8379
Epoch 4/25
593/593 [==============================] - 331s 558ms/step - loss: 0.3103 - accuracy: 0.8642 - val_loss: 0.2700 - val_accuracy: 0.8893
Epoch 5/25
593/593 [==============================] - 309s 521ms/step - loss: 0.2913 - accuracy: 0.8752 - val_loss: 0.2831 - val_accuracy: 0.8769
Epoch 6/25
593/593 [==============================] - 314s 530ms/step - loss: 0.2779 - accuracy: 0.8807 - val_loss: 0.2739 - val_accuracy: 0.8834
Epoch 7/25
593/593 [==============================] - 324s 546ms/step - loss: 0.2643 - accuracy: 0.8824 - val_loss: 0.2456 -

In [19]:
cnn.save("gender_model.h5")

In [27]:
import numpy as np
from keras.preprocessing import image as img
from keras.models import load_model
test_image = img.load_img('female_3500.jpg', target_size = (128, 128))
test_image = img.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
cnn=load_model('gender_model.h5')
result = cnn.predict(test_image)
print(training_set.class_indices)

if result[0][0] == 1:
    prediction = 'male'
else:
    prediction = 'female'

print(prediction)

{'female': 0, 'male': 1}
female
